In [53]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/u/shuhan/projects/vla')

from src.environments.highway_env.dataset import HighwayDataset, collate_fn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import torch
from torch.utils.data import DataLoader

data_folder = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action/rollouts'
dataset = HighwayDataset(data_folder, overfit=True)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [56]:
for batch in dataloader:
    break

In [55]:
from src.models.vlas.cont_obs_token_action import ContObsTokenActionVLA
from transformers import AutoModelForCausalLM, AutoTokenizer

llm_backbone = AutoModelForCausalLM.from_pretrained('gpt2')

hidden_dim = 768
obs_dim = 25
num_actions = 5
mlp_layers = 2

model = ContObsTokenActionVLA(llm_backbone, obs_dim, num_actions, hidden_dim, mlp_layers)

In [57]:
import os
# ckpt = os.path.expanduser('~/results/vla/quick_run/obs_reconst_only/lightning_logs/version_0/checkpoints/epoch=49-step=15650.ckpt')
ckpt = os.path.expanduser('~/results/vla/quick_run/action_obs_overfit_1_seq_from_obs_pt/lightning_logs/version_0/checkpoints/epoch=16-step=5321.ckpt')

lg_ckpt = torch.load(ckpt, map_location='cpu', weights_only=True)['state_dict']
ori_ckpt = {}
for k, v in lg_ckpt.items():
    if k.startswith('vla.'):
        ori_ckpt[k[4:]] = v

model.load_state_dict(ori_ckpt)

<All keys matched successfully>

In [64]:
ckpt

'/u/shuhan/results/vla/quick_run/action_obs_overfit_1_seq_from_obs_pt/lightning_logs/version_0/checkpoints/epoch=16-step=5321.ckpt'

In [59]:
outputs, predictions, loss_dict = model(*batch)

In [63]:
(predictions['action'].argmax(dim=-1) == batch[1]).float().mean()

tensor(0.8947)

In [43]:
outputs.keys()

odict_keys(['logits', 'past_key_values', 'hidden_states'])